In [1]:
fulldir="/Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/"
name="ppl2"

In [2]:
import cv2
from PIL import Image
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm
import numpy as np

In [3]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],image_size[1],color

In [4]:
image=fulldir+ name+"/"
ext='.bmp'
key = "abcdefghijklm"

In [5]:
def LogisticDecryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]

    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    
    L_x = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L_x *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1

    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    
    C = round((L_x*L_y*10**4) % 256)
    I_prev = C
    I_prev_r = C
    I_prev_g = C
    I_prev_b = C
    I = C
    I_r = C
    I_g = C
    I_b = C
    x_prev = 4*(S_x)*(1-S_x)
    y_prev = 4*(L_x)*(1-S_y)
    x = x_prev
    y = y_prev
    imageMatrix, dw, dh, color = getImageMatrix(imageName)

    logisticDecryptedImage = []
    for i in range(dw):
        row = []
        for j in range(dh):
            i_x=0
            while x <0.8 and x > 0.2 and i_x<=10:
                x = 4*x*(1-x)
                i_x=i_x+1
            i_y=0
            while y <0.8 and y > 0.2 and i_y<=10:
                y = 4*y*(1-y)
                i_y=i_y+1
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                I_r = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_r + key_list[7]) % N) ^ imageMatrix[i][j][0]) + N-key_list[6])%N
                I_g = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_g + key_list[7]) % N) ^ imageMatrix[i][j][1]) + N-key_list[6])%N
                I_b = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_b + key_list[7]) % N) ^ imageMatrix[i][j][2]) + N-key_list[6])%N
                I_prev_r = imageMatrix[i][j][0]
                I_prev_g = imageMatrix[i][j][1]
                I_prev_b = imageMatrix[i][j][2]
                row.append((I_r,I_g,I_b))
                x = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1  
            else:
                I = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev+key_list[7]) % N) ^ imageMatrix[i][j]) + N-key_list[6])%N
                I_prev = imageMatrix[i][j]
                row.append(I)
                x = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        logisticDecryptedImage.append(row)
    if color:
        im = Image.new("RGB", (dw, dh))
    else: 
        im = Image.new("L", (dw, dh)) # L is for Black and white pixels
    pix = im.load()
    for x in range(dw):
        for y in range(dh):
            pix[x, y] = henonDecryptedImage[x][y]
    print(imageName[28:-4])
    im.save(image+"decryp"+ext, "BMP")

In [6]:
def DecryptionResult(i, orig):
    #if i<10:
     #   imageName=fulldir+name+'/encrypted_frames/EncryptedResult00'+str(i)+'.bmp'
    #elif i<100:
     #   imageName=fulldir+name+'/encrypted_frames/EncryptedResult0'+str(i)+'.bmp'
    #else:
     #   imageName=fulldir+name+'/encrypted_frames/EncryptedResult'+str(i)+'.bmp'
    #im = cv2.cvtColor(np.array( Image.open(imageName)), cv2.COLOR_RGB2BGR)
    #print(type(im))
    #na = np.array(im)
    #orig = na.copy()
    na=orig #cv2.imread(imageName)
    print(na.shape)
    color_pixels= np.where(np.all(na==[255, 0, 0],axis=2))
    #print(color_pixels)
    
    zipped = np.column_stack((color_pixels[0], color_pixels[1]))

    #print(zipped)
    squares=[]
    for [y,x] in zipped:
        if np.array_equal(na[y+1,x], np.array([255, 0, 0]))==True and np.array_equal(na[y, x+1], np.array([255, 0, 0]))==True:
            #find the width and height
            w=0
            while (x+1+w)<na.shape[1] and  np.array_equal(na[y,x+1+w], np.array([255, 0, 0])):
                w=w+1
            h=0
            while (y+1+h)<na.shape[0] and np.array_equal(na[y+1+h,x], np.array([255, 0, 0])):
                h=h+1
            if w>1 and h>1:
                squares.append([x,y,w,h])
    print(squares)
    #print("frame: ", cnt)
    sum=0
    for [x,y,w,h] in squares:
        #print(a,b,c,d)
        cv2.imwrite(fulldir+name+"/hello.png", na[y:y+h+1, x:x+w+1])
        face=na[y+1:y+h,x+1:x+w]
        if len(face)==0:
            continue
        filename=image+"decryp"+ext
        cv2.imwrite(filename, face)
        LogisticDecryption(filename, key)
        filename=image+"decryp"+ext
        DecryptResult=cv2.imread(filename)
        na[y+1:y+h,x+1:x+w]=DecryptResult
        na[y,x]=np.mean(np.array([na[y-1, x], na[y, x-1]]),axis=0) 
        na[y+1:y+h,x]=np.mean(np.array([na[y+1:y+h, x-1], na[y+1:y+h, x+1]]),axis=0) 
        na[y+h, x]=np.mean(np.array([na[y+h+1, x], na[y+h, x-1]]),axis=0)
        na[y+h, x+1:x+w]=np.mean(np.array([na[y+h-1, x+1:x+w], na[y+h+1, x+1:x+w]]),axis=0) 
        na[y+h, x+w]=np.mean(np.array([na[y+h+1, x+w], na[y+h, x+w+1]]),axis=0)
        na[y+1:y+h, x+w]=np.mean(np.array([na[y+1:y+h, x+w-1], na[y+1:y+h, x+w+1]]),axis=0) 
        na[y, x+w]=np.mean(np.array([na[y-1, x+w], na[y, x+w+1]]),axis=0) 
        na[y, x+1:x+w]=np.mean(np.array([na[y-1, x+1:x+w], na[y+1, x+1:x+w]]),axis=0) 
        #diff = np.abs(orig[y+1:y+h, x+1:x+w].astype(np.float32) - na[y+1:y+h, x+1:x+w].astype(np.float32))
        #squares=np.square(diff)
        #sumofsquares = np.sum(squares)
        #rootofsquares = sumofsquares**0.5
        #loss=rootofsquares/3
        #sum+=loss
        #print(loss)
    #if(cnt<10): fname=fulldir+'SquareDetectionResults/decrypted_frames/decrypt0'+str(cnt)
    #else: fname=fulldir+'SquareDetectionResults/decrypted_frames/decrypt'+str(cnt)
    #cv2.imwrite(fname+ext, na)
    #cnt+=1
#    sum=sum/3
    return na

In [7]:
cam = cv2.VideoCapture(fulldir+name+"/results/encryptedvideo.avi")
fps = cam.get(cv2.CAP_PROP_FPS)
print(fps)
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
print(height, width)

30.0
1080 1920


In [8]:
import os

files=[]
entries = os.listdir(fulldir+name+'/encrypted_frames/')

for entry in entries:
    if entry[-4:]=='.bmp':
        files.append(entry)

files.sort()

print(files)

print(len(files))

['EncryptedResult000.bmp', 'EncryptedResult001.bmp', 'EncryptedResult002.bmp', 'EncryptedResult003.bmp', 'EncryptedResult004.bmp', 'EncryptedResult005.bmp', 'EncryptedResult006.bmp', 'EncryptedResult007.bmp', 'EncryptedResult008.bmp', 'EncryptedResult009.bmp', 'EncryptedResult010.bmp', 'EncryptedResult011.bmp', 'EncryptedResult012.bmp', 'EncryptedResult013.bmp', 'EncryptedResult014.bmp', 'EncryptedResult015.bmp', 'EncryptedResult016.bmp', 'EncryptedResult018.bmp', 'EncryptedResult019.bmp', 'EncryptedResult020.bmp', 'EncryptedResult022.bmp', 'EncryptedResult023.bmp', 'EncryptedResult024.bmp', 'EncryptedResult025.bmp', 'EncryptedResult026.bmp', 'EncryptedResult027.bmp', 'EncryptedResult028.bmp', 'EncryptedResult029.bmp', 'EncryptedResult030.bmp', 'EncryptedResult031.bmp', 'EncryptedResult032.bmp', 'EncryptedResult033.bmp', 'EncryptedResult034.bmp', 'EncryptedResult035.bmp', 'EncryptedResult036.bmp', 'EncryptedResult037.bmp', 'EncryptedResult038.bmp', 'EncryptedResult046.bmp', 'EncryptedR

In [9]:
video_name = 'decryptedvideo.avi'
print(fulldir+name+"/results/")
os.chdir(fulldir+name+"/results/")
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'FFV1'), fps, (width, height))
totalloss=0
loss=0
i=0
while(True):
    ret,Img = cam.read()
    temp=Img
    #if i<10:
     #   Img= cv2.imread(fulldir+name+"/frames/00"+str(i)+".png")
    #elif i<100:
     #   Img= cv2.imread(fulldir+name+"/frames/0"+str(i)+".png")
    #else:
     #   Img= cv2.imread(fulldir+name+"/frames/"+str(i)+".png")
    if ret==False:
        break
    temploss=0
    frameout=DecryptionResult(i, Img)
    totalloss+=temploss
    if i<10:
        cv2.imwrite(fulldir+name+"/decrypted_frames/00"+str(i)+".png", frameout)
    elif i<100:
        cv2.imwrite(fulldir+name+"/decrypted_frames/0"+str(i)+".png", frameout)
    else:
        cv2.imwrite(fulldir+name+"/decrypted_frames/"+str(i)+".png", frameout)
        #if currentframe<10:
            #cv2.imwrite(fulldir+"SquareDetectionResults/encrypted_frames_bmp/EncryptedResult0"+str(currentframe)+".bmp", frame)
        #else:
            #cv2.imwrite(fulldir+"SquareDetectionResults/encrypted_frames_bmp/EncryptedResult"+str(currentframe)+".bmp", frame)
    if i<10:
        Img= cv2.imread(fulldir+name+"/frames/00"+str(i)+".png")
    elif i<100:
        Img= cv2.imread(fulldir+name+"/frames/0"+str(i)+".png")
    else:
        Img= cv2.imread(fulldir+name+"/frames/"+str(i)+".png")
    i=i+1
    loss=loss+np.mean((frameout-Img)**2)
    print("frame: ", i)
    video.write(frameout)

/Users/mahitha0x01/Documents/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/results/
(1080, 1920, 3)
[[337, 152, 155, 209], [1374, 173, 145, 200], [873, 215, 144, 192]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
frame:  1
(1080, 1920, 3)
[[337, 152, 155, 209], [1374, 173, 145, 200], [873, 215, 144, 193]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
frame:  2
(1080, 1920, 3)
[[336, 152, 156, 209], [1374, 172, 145, 201], [872, 215, 145, 193]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
frame:  3
(1080, 1920, 3)
[[336, 152, 

/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
frame:  28
(1080, 1920, 3)
[[336, 152, 156, 209], [1374, 172, 144, 201], [919, 200, 136, 191]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
frame:  29
(1080, 1920, 3)
[[336, 152, 156, 209], [1374, 172, 144, 200], [925, 197, 132, 194]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
frame:  30
(1080, 1920, 3)
[[336, 152, 156, 209], [1374, 172, 144, 200], [927, 197, 132, 191]]
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
/svnit/Assignments/FYP/Secure-Sharing-of-CCTV-footage/ppl2/decryp
frame:  31
(1080, 1920, 3)
[[336, 152, 156, 209], [1374

In [10]:
video.release() 
print(totalloss/len(files))
print(loss/len(files))

0.0
6.030033967928934


In [11]:
print(i)

52


In [12]:
print(loss/(i-1))

6.148269928084403
